One of the applications of convex optimization in complex variables is phase recovery, a problem in signal processing. The objective is to reconstruct the complex phase of a vector when we are only given the magnitude of some linear measurements.

This problem is orginally a nonconvex optimization problem. Using results from [this paper](https://arxiv.org/abs/1206.0102), the problem can be relaxed to a (complex) semidefinite program (complex SDP).

Given a linear operator $A$ and a vector $b$ of measured amplitudes, define the positive semidefinite hermitian matrix $M = \text{diag}(b) (I - A A^*) \text{diag}(b)$. The problem is:

                minimize < U,M >
                subject to 
                diag(U) = 1
                U in :HermitianSemiDefinite
                
Here the variable $U$ must be hermitian ($U \in \mathbb{H}_n $), and we have a solution to the phase recovery problem if $U = u u^*$ has rank one. Otherwise, the leading singular vector of $U$ can be used to approximate the solution.


In [2]:
using Convex

n = 5
M = rand(n,n) + im*rand(n,n)
U = ComplexVariable(n,n)
objective = inner_product(U,M)
c1 = diag(U) == 1 
c2 = U in :SDP
p = minimize(objective,c1,c2)
solve!(p)
U.value

(size(coeff),size(var)) = ((10,10),(10,10))
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 201
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 51, constraints m = 111
Cones:	primal zero / dual free vars: 56
	sd vars: 55, sd blks: 1
Setup time: 1.85e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  1.55e-03 
   100| 2.28e-03  3.22e-02  1.36e-03 -3.00e+00 -3.01e+00  1.18e-16  7.47e-03 
   200| 8.63e-05  2.61e-03  6.40e-08 -3.00e+00 -3.00e+00  4.93e-17  1.63e-02 
   

5×5 Array{Complex{Float64},2}:
 1.0+3.58937e-9im             0.0857607-0.996316im    -0.995607-0.0936257im      0.973338+0.229375im     -0.946793+0.321843im
    0.0857607+0.996316im   1.0+2.51013e-11im         0.00789686-0.999969im      -0.145056+0.989423im     -0.401855-0.915703im
    -0.995607+0.0936257im    0.00789686+0.999969im  1.0-2.9097e-9im             -0.990538-0.137238im      0.912501-0.409073im
     0.973338-0.229375im      -0.145056-0.989423im    -0.990538+0.137238im   1.0+2.49128e-9im            -0.847727+0.530433im
    -0.946793-0.321843im      -0.401855+0.915703im     0.912501+0.409073im      -0.847727-0.530433im  1.0-2.24718e-9im       